<a href="https://colab.research.google.com/github/surkovvv/loan_default_prediction/blob/nick/LoanDefaultPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Первая подзадача на 18-20.10.21
1) Разобраться с основным dataset'ом из работы

2) Разобраться аналогично с еще 2мя датасетами : UCI и CS

3) Почитать статью из кэггла про успешные модели в соревнованиях - там че-то странное, говорят про удачные комбинации фичей

4) Посмотреть как работают на этих данных RF, XGBoost, CatBoost, LightGMB

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
%matplotlib inline

sns.set(style="darkgrid")

In [ ]:
df1 = pd.read_csv("applicationdata.csv")
df1.sample(10)
pd.options.display.max_columns = None
df1.head(30)
№df1["Reason"].nunique()


895

Посмотрим на тех, кто несколько раз обращался за кредитом (кол-во уникальных CustomerID меньше кол-ва объектов, значит, такие люди есть)

# Отступ ⬇

# Мини-план, что вообще делаем:
### 1)Опять посмотрим на признаки : у нас есть некоторые признаки, у которых совпадают префиксы, для начала посмотрим на них, на их пропуски, на их совместные распределения(или сопоставления). Сделаем выводы, поудаляем или скрестим некоторые признаки(лучше второе)

### 2) Попробуем сделать так: сначала обучаем модель на небольшом количестве признаков, смотрим на качество, добавляем какие-то признаки (группками или по одному), смотрим на качество. Естественно, мы хотим, чтобы качество росло. Но в таком подходе есть минус - модель может быть тяжеловесной - например, любой из бустингов, и тогда этот процесс может затянуться до бесконечности... Решением этой проблемы может быть, например, использование LogisticRegression

### 3) Для осуществления пункта 2, для начала выберем те самые признаки, которые изначально уже будут в таблице (скорее всего, это почти все признаки с адекватными названиями + те признаки, которые мы получим с 1 пункта)

### 4) В крайнем случае придется прибегнуть к глубокому анализу признаков - визаулизировать некоторые зависимости, посмотреть на таблицу корреляции или еще чего мощнее.

### 5) Когда данные будут готовы, мы будем подбирать гиперпараметры для наших базовых моделей с помощью кросс-валидации (число фолдов тоже будет гиперпараметром)

### 6) Если всё пройдет более-менее гладко, мы должны в конце концов победить 🥇🥇🥇🥇🥇

# Отспуп ⬆

In [5]:
df1.head(2)

,CustomerID,LoanType,Reason,Application.Date,Employment.Length,Employment.Sector,RequestedAmount,OfferedAmount,FundedAmount,Duration,Age,EmailDomain,Residential.Status,Sex,Marital.Status,Dependents,City,State,PostalCode,Pay.Frequency,Product,Risk.Rating,Employment.Type,Requested.Duration,ArrearsLevel,IsWrittenOff,age.of.file,age.of.subject,time.at.address,time.at.employer,defaults,total.value.of.outstanding.defaults,defaults.paid,defaults.12,time.since.last.default,credit.enquiries.1,credit.enquiries.3,credit.enquiries.6,credit.enquiries.12,credit.enquiries.60,time.since.last.enquiry,telco.and.utility.defaults,telco.and.utility.defaults.12,telco.and.utility.enquiries,telco.and.utility.enquiries.6,telco.and.utility.enquiries.12,directorships.current,directorships.previous,adverse.on.file,applicant.odds,veda.score,contributing.factors,Wages_num,Centrelink_num,SACC_num,NonSACC_num,DebtConsolidation_num,DebtCollection_num,Rent_num,Overdrawn_num,Uncategorised_Debits_num,Other_Debits_num,All_Other_Credits_num,Utilities_num,ATM_num,Gambling_num,Groceries_num,Insurance_num,Subscription_TV,Telecommunications_num,Transport_num,Dishonours_num,Tax_num,currentBalance_All,availableBalance_All,openingBalance_All,totalCredits_All,totalDebits_All,closingBalance_All,daysInNegative_All,currentBalance_CC,availableBalance_CC,openingBalance_CC,totalCredits_CC,totalDebits_CC,currentBalance_PL,availableBalance_PL,openingBalance_PL,totalCredits_PL,totalDebits_PL,currentBalance_Tran,availableBalance_Tran,openingBalance_Tran,totalCredits_Tran,totalDebits_Tran,closingBalance_Tran,daysInNegative_Tran,totalTX_All,totalCR_All,totalDR_All,totalCRvalue_All,totalDRvalue_All,maxCRvalue_All,minCRvalue_All,maxDRvalue_All,minDRvalue_All,minBalance_All,maxBalance_All,totalTX_CC,totalCR_CC,totalDR_CC,totalCRvalue_CC,totalDRvalue_CC,maxCRvalue_CC,minCRvalue_CC,maxDRvalue_CC,minDRvalue_CC,minBalance_CC,maxBalance_CC,totalCRvalue_PL,totalDRvalue_PL,maxCRvalue_PL,minCRvalue_PL,maxDRvalue_PL,minDRvalue_PL,minBalance_PL,maxBalance_PL,totalCRvalue_Tran,totalDRvalue_Tran,maxCRvalue_Tran,minCRvalue_Tran,maxDRvalue_Tran,minDRvalue_Tran,minBalance_Tran,maxBalance_Tran,EntropyText,TDatePart,EntropyDate,Salary,Rent,LoanCommitments,LivingExpenses,Wages,Centrelink,SaccLoans,NonSaccLoans,DebtConsolidation,DebtCollection,Rent1,OverdrawnFees,UncategorisedDebits,OtherDebits,AllOtherCredits,Utilities,AtmWithdrawals,Gambling,Groceries,Insurance,SubscriptionTv,Telecommunications,Transport,OnGoingSaccCount,DishounorIn90Days,Income,NonDiscretionarySpending,DiscretionarySpending,RepaymentCapacity,RepaymentFromRequested,ExcessCapacity,RepaymentsRatio,Centrelink_wages_ratio,SaccLoans_wages_ratio,NonSaccLoans_wages_ratio,DebtConsolidation_wages_ratio,DebtCollection_wages_ratio,Rent1_wages_ratio,OverdrawnFees_wages_ratio,UncategorisedDebits_wages_ratio,OtherDebits_wages_ratio,AllOtherCredits_wages_ratio,Utilities_wages_ratio,AtmWithdrawals_wages_ratio,Gambling_wages_ratio,Groceries_wages_ratio,Insurance_wages_ratio,SubscriptionTv_wages_ratio,Telecommunications_wages_ratio,Transport_wages_ratio,Wages_repaymentcapacity_ratio,Centrelink_repaymentcapacity_ratio,SaccLoans_repaymentcapacity_ratio,NonSaccLoans_repaymentcapacity_ratio,DebtConsolidation_repaymentcapacity_ratio,DebtCollection_repaymentcapacity_ratio,Rent1_repaymentcapacity_ratio,OverdrawnFees_repaymentcapacity_ratio,UncategorisedDebits_repaymentcapacity_ratio,OtherDebits_repaymentcapacity_ratio,AllOtherCredits_repaymentcapacity_ratio,Utilities_repaymentcapacity_ratio,AtmWithdrawals_repaymentcapacity_ratio,Gambling_repaymentcapacity_ratio,Groceries_repaymentcapacity_ratio,Insurance_repaymentcapacity_ratio,SubscriptionTv_repaymentcapacity_ratio,Telecommunications_repaymentcapacity_ratio,Transport_repaymentcapacity_ratio,Income_MonthlyRepayments_ratio,NonDiscretionarySpending_MonthlyRepayments_ratio,DiscretionarySpending_MonthlyRepayments_ratio,RepaymentCapacity_MonthlyRepayments_ratio,MaxCdia,Isdefault,Isapproved,Inrange,Hasjson
0,161181.0

In [ ]:
# Это нужно перетащить пониже, в раздел, где я буду разбираться с уникальными ID
vc = df1["CustomerID"].value_counts()
mask = vc > 1
val = vc[mask]
another_mask = df1["CustomerID"].isin(val.index.values)
(df1[another_mask].sort_values('CustomerID')["Isdefault"])

2497     0.0
8471     0.0
680      0.0
8405     0.0
4028     0.0
        ... 
12021    0.0
9534     0.0
7644     0.0
9782     0.0
9252     0.0
Name: Isdefault, Length: 1345, dtype: float64

In [ ]:
#df1.head()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12982 entries, 0 to 12981
Columns: 214 entries, CustomerID to Hasjson
dtypes: float64(192), object(22)
memory usage: 21.2+ MB


In [ ]:
features = df1.columns.values
features.shape[0]

214

Видно, что признаков очень много, но возникает ощущение, что далеко не все из них полезны

In [30]:
df1 = df1.drop(index=df1[df1.isna().sum(axis=1) >= 100].index)

In [11]:
X = df1.drop("Isdefault", axis=1)
y = df1["Isdefault"]

In [12]:
y = y.iloc[:-2].astype(int)
X = X.iloc[:-2, :]

In [ ]:
for col_name in df1.columns:
    print("{} ---- {} ".format(col_name, df1[col_name].nunique()))

In [ ]:
#много уникальных значений, лучше удалить сразу


# Плохая ячейка, неясно, почему только эти, а другие нет


# X = X.drop(columns=["CustomerID", "Application.Date", "TDatePart", "DebtCollection_num", "contributing.factors"])
# X.loc[X["time.since.last.default"].isna(), "time.since.last.default"] = 0 # тут nan это скорее значит, что дефолта не было

In [ ]:
for col in X:
    print(col, X[col].isna().sum())

#### ***Видно***, что всякие признаки, названия которых оканчиваются на num, CC, PL, Tran и ratio имеют непривлекательно много пропусков, к тому же, для CC, PL, Tran префиксы совпадают для признаков, оканчивающихся на All, которые мы и возьмем в нашу модель, удалим их

In [14]:
bad_features = []
for col_name in df1.columns:
    if (col_name.endswith("num") or col_name.endswith("_CC") or col_name.endswith("_PL") or 
        col_name.endswith("_Tran") or col_name.endswith("y_ratio")):
        bad_features.append(col_name)
X.drop(columns=bad_features, inplace=True)

## Мое предположение, что от адреса электронной почты, postal code и applycationDate таргет слабо зависит, можно их дропнуть(сделаем из даты 2 новых столбца - год и месяц), а так же удалим признаки, способствующих переобучению (по заявлению автора статьи) IsWrittenOff, MaxCdia, а так же последние три признака - у них всего 1 уникальное значение, это ни о чем не скажет нашей модели

## Второе замечание - уникальных ID всего 12300, надо с ними разобраться..

In [25]:
X.drop(columns=["PostalCode", "IsWrittenOff", "MaxCdia", "Isapproved", "Inrange", "Hasjson"], inplace=True)

Посмотрим пристально на категориальные данные:

In [27]:
for col in X.select_dtypes(include='object').columns.values:
    print(col, X[col].isna().sum(), X[col].nunique())
# Удалим contributing.factors, TDatePart, ApplicationDate разобьем на год и месяц
X.drop(columns=["contributing.factors", "TDatePart"], inplace=True)

LoanType 0 2
Reason 0 895
Application.Date 0 12978
Employment.Length 47 3
Employment.Sector 57 26
Duration 0 34
EmailDomain 0 557
Residential.Status 7 3
Sex 0 2
Marital.Status 136 3
Dependents 136 4
City 2 4181
State 0 9
Pay.Frequency 0 3
Product 0 2
Risk.Rating 0 7
Employment.Type 0 5
Requested.Duration 0 39
ArrearsLevel 5899 4
adverse.on.file 157 4
contributing.factors 157 517
TDatePart 0 12976


### Выудим из даты год и месяц, из причин, города, емеила - самые популярные 

In [29]:
for col in X.select_dtypes(include='number').columns.values:
    print(col, X[col].isna().sum(), X[col].nunique())

CustomerID 0 12300
RequestedAmount 0 95
OfferedAmount 0 86
FundedAmount 0 87
Age 0 48
age.of.file 157 2690
age.of.subject 157 2618
time.at.address 157 1758
time.at.employer 1114 908
defaults 157 3
total.value.of.outstanding.defaults 2984 259
defaults.paid 157 3
defaults.12 157 3
time.since.last.default 12369 128
credit.enquiries.1 157 9
credit.enquiries.3 157 17
credit.enquiries.6 157 21
credit.enquiries.12 157 28
credit.enquiries.60 157 50
time.since.last.enquiry 681 902
telco.and.utility.defaults 157 3
telco.and.utility.defaults.12 157 3
telco.and.utility.enquiries 157 12
telco.and.utility.enquiries.6 157 5
telco.and.utility.enquiries.12 157 6
directorships.current 157 7
directorships.previous 157 8
applicant.odds 157 936
veda.score 112 684
Subscription_TV 9857 19
currentBalance_All 5 12681
availableBalance_All 5 11153
openingBalance_All 5 12541
totalCredits_All 5 12942
totalDebits_All 5 12954
closingBalance_All 5 12691
daysInNegative_All 5 271
totalTX_All 10 973
totalCR_All 10 354
t

## Итоги: 

### Миша написал выше функцию для Reason, я отобрал почти все нужные столбцы, осталось разобраться с дубликатами в ID, доделать функции для энкодинга категориальных признаков, придумать что делать с пропусками

In [ ]:
columns_to_delete = []
columns_to_average = []
for col in X.columns:
    tmp = X[col].isna()
    #print(tmp.value_counts().index.values)
    if len(tmp.value_counts().index.values) == 2 or True in tmp.value_counts().index.values:
        #print(tmp.value_counts().loc[False])
        print(col, ":", tmp.value_counts().loc[True], "/", X.shape[0], tmp.value_counts().loc[True] / X.shape[0], end=", ")
        if  tmp.value_counts().loc[True] / X.shape[0] > 0.05:
            print("=> to delete")
            columns_to_delete.append(col)
        else:
            print("=> to average")
            columns_to_average.append(col)

Employment.Length : 47 / 12980 0.00362095531587057, => to average
Employment.Sector : 57 / 12980 0.004391371340523883, => to average
Residential.Status : 7 / 12980 0.0005392912172573189, => to average
Marital.Status : 136 / 12980 0.010477657935285054, => to average
Dependents : 136 / 12980 0.010477657935285054, => to average
City : 2 / 12980 0.00015408320493066256, => to average
ArrearsLevel : 5899 / 12980 0.45446841294298923, => to delete
age.of.file : 157 / 12980 0.01209553158705701, => to average
age.of.subject : 157 / 12980 0.01209553158705701, => to average
time.at.address : 157 / 12980 0.01209553158705701, => to average
time.at.employer : 1114 / 12980 0.08582434514637904, => to delete
defaults : 157 / 12980 0.01209553158705701, => to average
total.value.of.outstanding.defaults : 2984 / 12980 0.22989214175654854, => to delete
defaults.paid : 157 / 12980 0.01209553158705701, => to average
defaults.12 : 157 / 12980 0.01209553158705701, => to average
credit.enquiries.1 : 157 / 12980 

In [ ]:
X["Reason"] = X["Reason"].apply(lambda x : x.replace(",", " ").split()[0])

Было бы неплохо придумать, как забить столько пропусков

Ясно, что некоторые модели хорошо работают и с ними, но тот же logres не справляется - выдает ошибку далее... его можно вообще не использовать - чтобы не мучаться, но можно и подумать над этим

Мое решение - если это категориальный признак - заменить пропуски самым частым классом

Если числовой - то можно для начала заменить средними с шумом

Но для начала удалим слишком плохие признаки - у них будет больше 5 % пустот

## Сделано

In [ ]:
X.head()

,Reason,RequestedAmount,OfferedAmount,FundedAmount,Duration,Age,EmailDomain,City,PostalCode,Requested.Duration,age.of.file,age.of.subject,time.at.address,defaults,defaults.paid,defaults.12,time.since.last.default,credit.enquiries.1,credit.enquiries.3,credit.enquiries.6,credit.enquiries.12,credit.enquiries.60,telco.and.utility.defaults,telco.and.utility.defaults.12,telco.and.utility.enquiries,telco.and.utility.enquiries.6,telco.and.utility.enquiries.12,directorships.current,directorships.previous,applicant.odds,veda.score,All_Other_Credits_num,ATM_num,currentBalance_All,availableBalance_All,openingBalance_All,totalCredits_All,totalDebits_All,closingBalance_All,daysInNegative_All,...,Residential.Status_Rent,Sex_F,Sex_M,Marital.Status_Defacto,Marital.Status_Married,Marital.Status_Single,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,State_Australian Capital Territory,State_New South Wales,State_Northern Territory,State_Other,State_Queensland,State_South Australia,State_Tasmania,State_Victoria,State_Western Australia,Pay.Frequency_Fortnightly,Pay.Frequency_Monthly,Pay.Frequency_Weekly,Product_MACC,Product_PL,Risk.Rating_A1,Risk.Rating_A1+,Risk.Rating_A11,Risk.Rating_A2,Risk.Rating_A3,Risk.Rating_A4,Risk.Rating_A5,Employment.Type_Centrelink Only,Employment.Type_Full time,Employment.Type_Other,Employment.Type_Part time,Employment.Type_Self-employed,adverse.on.file_N,adverse.on.file_No,adverse.on.file_Y,adverse.on.file_Yes
0,29,-0.991067,-0.702408,-0.702232,2,-1.032036,295,738,-0.971826,18,-0.583886,-0.543921,-0.412636,4.184726,-0.152158,11.835398,0.408689,0.448548,1.298805,1.107089,1.065069,1.194854,4.184726,11.835398,1.170383,-0.31962,-0.43691,-0.161187,-0.112921,-0.871035,-2.647754,-0.783234,1.662500e-16,0.235090,0.002610,0.187319,-0.417475,0.242107,0.214609,-0.666270,...,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
1,12,0.128265,-0.415201,-0.415059,14,-0.557047,233,2476,-1.991541,18,-0.215768,-0.519776,-0.115643,-0.220195,-0.152158,-0.081784,-0.198364,-0.619554,-0.804754,-0.945720,-0.366935,0.277125,-0.220195,-0.081784,-0.723827,-0.31962,-0.43691,-0.161187,-0.112921,-0.601300,-0.771891,-0.699959,-7.104488e-01,0.246976,0.106143,0.199691,0.013397,-0.192732,0.226542,-0.643738,...,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
2,20,0.426753,1.480361,1.480286,21,0.487929,207,835,-0.623032,32,1.430542,-0.447342,0.107101,-0.220195,-0.152158,-0.081784,-0.198364,-0.619554,0.597619,0.593887,-0.008934,1.194854,-0.220195,-0.081784,1.170383,-0.31962,-0.43691,-0.161187,-0.112921,-0.304593,-0.026461,-0.117029,1.863288e+00,0.232317,-0.011593,0.187968,-0.179851,-0.027817,0.211825,-0.621206,...,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0
3,6,1.807262,0.733624,0.733635,14,-0.747043,207,1444,-0.617418,32,-0.399827,-0.525813,-0.264140,-0.220195,-0.152158,-0.081784,-0.198364,0.448548,-0.103568,0.080684,0.349067,1.719271,-0.220195,-0.081784,1.170383,-0.31962,-0.43691,-0.161187,-0.112921,-0.666037,-1.017637,-0.766579,-8.508344e-01,0.226987,-0.079815,0.187319,-0.422938,0.178633,0.206474,-0.666270,...,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0
4,12,-0.244846,-0.702408,-0.702232,3,-0.177056,362,667,-1.948029,3,-0.880427,-0.489595,-0.189891,-0.220195,-0.152158,-0.081784,-0.198364,-0.619554,0.597619,0.593887,-0.008934,-0.509500,-0.220195,-0.081784,-0.723827,-0.31962,-0.43691,-0.161187,-0.112921,-0.703799,-1.197851,-0.649993,-2.892919e-01,0.223232,-0.083376,0.187347,-0.437025,0.185375,0.202704,-0.350820,...,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0


In [ ]:
X = X.drop(columns=columns_to_delete)
for column in columns_to_average:
    if column in X.select_dtypes(include='number').columns.values:
        col_mean = X[column].mean()
        X[column][X[column].isna()] = col_mean * np.ones(X[column][X[column].isna()].index.values.shape[0])
    elif column in X.select_dtypes(include='object').columns.values:
        col_mode = X[column].mode().values
        X[column][X[column].isna()] = [col_mode for _ in range(X[column].isna().index.values.shape[0])]

__Итого:__ Выделили отдельно таргет и дату, Reason'ов около 900, никакого OHE на него вешать не стоит, может быть стоит удалить колонку с Id, loanType явно может сказать что-то полезное, дату тоже можно оставить, но переделать ее в pd.datetime, emp.len перекодируем labelEncoder'ом, Emp.Sec можно OHE, duration тоже label, или просто срезать месяцы(по первым 2м символам строки)

In [ ]:
pip install lightgbm

In [ ]:
pip install catboost

In [ ]:
pip install xgboost

In [ ]:
pip install -U KTBoost

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import precision_recall_curve, classification_report, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier
import xgboost as xgb
import KTBoost.KTBoost as KTBoost

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=66)
numeric_features = X.select_dtypes(include='number').columns
categorical_features = X.select_dtypes(include='object').columns
ohe_categorical = []
indicies = []
for i, cat in enumerate(categorical_features):
    if (X[cat].nunique() < 30):
        ohe_categorical.append(cat)
        indicies.append(i)
categorical_features = np.delete(categorical_features.values, indicies)
print(categorical_features)
print(ohe_categorical)

scaler = StandardScaler()
encoder = LabelEncoder()

ohe_cat_df = pd.get_dummies(X[ohe_categorical])
X = X.drop(columns=ohe_categorical)
for column in categorical_features:
    X[column] = encoder.fit_transform(X[column])
X[numeric_features] = scaler.fit_transform(X[numeric_features])
X = X.join(ohe_cat_df)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=66, stratify=y)

['Reason' 'Duration' 'EmailDomain' 'City' 'Requested.Duration']
['LoanType', 'Employment.Length', 'Employment.Sector', 'Residential.Status', 'Sex', 'Marital.Status', 'Dependents', 'State', 'Pay.Frequency', 'Product', 'Risk.Rating', 'Employment.Type', 'adverse.on.file']


In [ ]:
X.head()

,Reason,RequestedAmount,OfferedAmount,FundedAmount,Duration,Age,EmailDomain,City,PostalCode,Requested.Duration,age.of.file,age.of.subject,time.at.address,defaults,defaults.paid,defaults.12,time.since.last.default,credit.enquiries.1,credit.enquiries.3,credit.enquiries.6,credit.enquiries.12,credit.enquiries.60,telco.and.utility.defaults,telco.and.utility.defaults.12,telco.and.utility.enquiries,telco.and.utility.enquiries.6,telco.and.utility.enquiries.12,directorships.current,directorships.previous,applicant.odds,veda.score,All_Other_Credits_num,ATM_num,currentBalance_All,availableBalance_All,openingBalance_All,totalCredits_All,totalDebits_All,closingBalance_All,daysInNegative_All,...,Residential.Status_Rent,Sex_F,Sex_M,Marital.Status_Defacto,Marital.Status_Married,Marital.Status_Single,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,State_Australian Capital Territory,State_New South Wales,State_Northern Territory,State_Other,State_Queensland,State_South Australia,State_Tasmania,State_Victoria,State_Western Australia,Pay.Frequency_Fortnightly,Pay.Frequency_Monthly,Pay.Frequency_Weekly,Product_MACC,Product_PL,Risk.Rating_A1,Risk.Rating_A1+,Risk.Rating_A11,Risk.Rating_A2,Risk.Rating_A3,Risk.Rating_A4,Risk.Rating_A5,Employment.Type_Centrelink Only,Employment.Type_Full time,Employment.Type_Other,Employment.Type_Part time,Employment.Type_Self-employed,adverse.on.file_N,adverse.on.file_No,adverse.on.file_Y,adverse.on.file_Yes
0,29,-0.991067,-0.702408,-0.702232,2,-1.032036,295,738,-0.971826,18,-0.583886,-0.543921,-0.412636,4.184726,-0.152158,11.835398,0.408689,0.448548,1.298805,1.107089,1.065069,1.194854,4.184726,11.835398,1.170383,-0.31962,-0.43691,-0.161187,-0.112921,-0.871035,-2.647754,-0.783234,1.662500e-16,0.235090,0.002610,0.187319,-0.417475,0.242107,0.214609,-0.666270,...,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
1,12,0.128265,-0.415201,-0.415059,14,-0.557047,233,2476,-1.991541,18,-0.215768,-0.519776,-0.115643,-0.220195,-0.152158,-0.081784,-0.198364,-0.619554,-0.804754,-0.945720,-0.366935,0.277125,-0.220195,-0.081784,-0.723827,-0.31962,-0.43691,-0.161187,-0.112921,-0.601300,-0.771891,-0.699959,-7.104488e-01,0.246976,0.106143,0.199691,0.013397,-0.192732,0.226542,-0.643738,...,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
2,20,0.426753,1.480361,1.480286,21,0.487929,207,835,-0.623032,32,1.430542,-0.447342,0.107101,-0.220195,-0.152158,-0.081784,-0.198364,-0.619554,0.597619,0.593887,-0.008934,1.194854,-0.220195,-0.081784,1.170383,-0.31962,-0.43691,-0.161187,-0.112921,-0.304593,-0.026461,-0.117029,1.863288e+00,0.232317,-0.011593,0.187968,-0.179851,-0.027817,0.211825,-0.621206,...,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0
3,6,1.807262,0.733624,0.733635,14,-0.747043,207,1444,-0.617418,32,-0.399827,-0.525813,-0.264140,-0.220195,-0.152158,-0.081784,-0.198364,0.448548,-0.103568,0.080684,0.349067,1.719271,-0.220195,-0.081784,1.170383,-0.31962,-0.43691,-0.161187,-0.112921,-0.666037,-1.017637,-0.766579,-8.508344e-01,0.226987,-0.079815,0.187319,-0.422938,0.178633,0.206474,-0.666270,...,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0
4,12,-0.244846,-0.702408,-0.702232,3,-0.177056,362,667,-1.948029,3,-0.880427,-0.489595,-0.189891,-0.220195,-0.152158,-0.081784,-0.198364,-0.619554,0.597619,0.593887,-0.008934,-0.509500,-0.220195,-0.081784,-0.723827,-0.31962,-0.43691,-0.161187,-0.112921,-0.703799,-1.197851,-0.649993,-2.892919e-01,0.223232,-0.083376,0.187347,-0.437025,0.185375,0.202704,-0.350820,...,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0


In [ ]:
logres = LogisticRegression()
#logres_pipeline = Pipeline([
#    ('transforming', col_transformer),
#    ('LogisticRegression', logres)
#])
logres.fit(X_train, y_train)
y_pred1 = logres.predict(X_test)
#print("Accuracy on LogisticRegression=", accuracy_score(y_test, y_pred1))
print(classification_report(y_test, y_pred1))

              precision    recall  f1-score   support

           0       0.83      0.99      0.90      2673
           1       0.41      0.03      0.06       572

    accuracy                           0.82      3245
   macro avg       0.62      0.51      0.48      3245
weighted avg       0.75      0.82      0.75      3245



In [ ]:
model = KTBoost.BoostingClassifier(loss='deviance',base_learner='combined',
                                    update_step='newton',theta=1)
model.fit(X_train, y_train)
y_pred_grabit = model.predict(X_test)
print(classification_report(y_test, y_pred_grabit))

TypeError: ignored

In [ ]:
model = KTBoost.BoostingClassifier(loss='deviance',update_step='gradient')
model.fit(X_train, y_train)
y_pred_g = model.predict(X_test)
print(classification_report(y_test, y_pred_g))

In [ ]:
AdaBoost = AdaBoostClassifier(n_estimators=100, random_state=66)
AdaBoost.fit(X_train, y_train)
y_pred2 = AdaBoost.predict(X_test)
#print("Accuracy on AdaBoost Classifier=", accuracy_score(y_test, y_pred2))
print(classification_report(y_test, y_pred2))

In [ ]:
GradBoost = GradientBoostingClassifier(n_estimators=100, random_state=66)
GradBoost.fit(X_train, y_train)
y_pred3 = GradBoost.predict(X_test)
#print("Accuracy on Bradient Boosting Classifier=", accuracy_score(y_test, y_pred3))
print(classification_report(y_test, y_pred3))

In [ ]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
y_pred4 = lgbm.predict(X_test)
print("Accuracy on LGBM Classifier=", accuracy_score(y_test, y_pred4))
#print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

LGBM_GetLastError().decode('utf-8') - попробуй декодировать по всем столбцам и выявить плохой - и переименовать - почему то не работает

In [ ]:
catBoost = CatBoostClassifier()
catBoost.fit(X_train,y_train,verbose=False)#, plot=True)
y_pred5 = catBoost.predict(X_test)
print("Accuracy on CatBoost Classifier=", accuracy_score(y_test, y_pred5))
#catBoost.score(X_test, y_test)
# тут походу можно просто запихать df изначальный и все скушает

In [ ]:
# небольшой эксперимент с catboost'ом : попробую запихать туда необработанный датасет
model_params = {
    'iterations': 500, 
    'loss_function': 'Logloss', 
    'train_dir': 'crossentropy',
    'allow_writing_files': False,
    'random_seed': 66,
}

cat_clf = CatBoostClassifier(**model_params)

X_ = df1.drop("Isdefault", axis=1)
y_ = df1["Isdefault"]
y_ = y.iloc[:-2].astype(int)
X_ = X.iloc[:-2, :]

X_train_, X_test_, y_train_, y_test_ = train_test_split(X_, y_, train_size=0.8, random_state=66)
cat_clf.fit(X_train_, y_train_)
y_pred_ = cat_clf.predict(X_test_)
print("Accuracy score on raw data with CatBoost Classifier:", accuracy_score(y_test_, y_pred_))
print("Как видно, точность всё таки ниже, чем на обработанных данных => мы сделали предобработку данных неплохо")

In [ ]:
xg_cl = xgb.XGBClassifier(colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_cl.fit(X_train,y_train)

y_pred6 = xg_cl.predict(X_test)
#print("Accuracy on XGBoost Classifier=", accuracy_score(y_test, y_pred6))
print(classification_report(y_test, y_pred6))

In [ ]:
rf_cl = RandomForestClassifier()
rf_cl.fit(X_train,y_train)

y_pred7 = rf_cl.predict(X_test)
#print("Accuracy on RandomForest Classifier=", accuracy_score(y_test, y_pred7))
print(classification_report(y_test, y_pred7))

# Абстрагирование от данного датасета

In [ ]:
data = pd.read_csv('UCI_Credit_Card.csv')
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [ ]:
data.drop(columns=['ID'], inplace=True)

In [ ]:
data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [ ]:
# хреновый датасет, оч мало признаков

In [ ]:
df = pd.read_csv("train_v2.csv")
df.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,...,f740,f741,f742,f743,f744,f745,f746,f747,f748,f749,f750,f751,f752,f753,f754,f755,f756,f757,f758,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
0,1,126,10,0.686842,1100,3,13699,7201.0,4949.0,126.75,126.03,7,0.7607,0.7542,612922,0.7236,0.7236,0.5171,0.7236,0.8476,0.7876,1.097851e+09,89,66,998046.0,89.0,89.0,89.00,89.00,89.0,0.9271,0,0,0,5,0,0,0.73180,0.72124,0.749280,...,-4.40,6,9.76,-21.82,49.18,1.0435,-0.9066,1.5883,3.4154,2.8283,2.3451,0.030594,1.7418,1.5271,0.8474,0.4715,0.028362,3.1611,2.5162,2.0037,0.019636,4.4352,4.2676,-0.1524,1,-0.40,-0.560,-0.440,-0.6280,-3.14,5,2.14,-1.54,1.18,0.1833,0.7873,1,0,5,0
1,2,121,10,0.782776,1100,3,84645,240.0,1625.0,123.52,121.35,7,0.6555,0.6555,245815,0.6341,0.6341,0.3909,0.6667,0.6903,0.6903,8.449459e+08,78,50,754416.0,78.0,78.0,78.00,78.00,78.0,0.8478,0,0,0,6,0,0,0.65705,0.64069,0.626200,...,0.35,7,51.73,-72.44,798.74,2.7180,-0.8294,1.0740,2.9194,2.1180,1.5666,0.120442,1.1963,1.0322,0.4843,0.2389,0.130160,2.7659,1.9523,1.4059,0.115277,3.2763,2.7962,-0.3097,1,-0.17,-0.275,-0.203,-0.2300,-1.38,6,0.54,-0.24,0.13,0.1926,-0.6787,1,0,5,0
2,3,126,10,0.500080,1100,3,83607,1800.0,1527.0,127.76,126.49,7,0.7542,0.7542,1385872,0.7542,0.7542,0.5508,0.7542,0.8091,0.7807,1.308478e+09,89,54,1037651.0,89.0,89.0,100.43,94.37,89.0,0.9175,0,0,0,13,0,0,0.74710,0.75878,0.654685,...,-6.35,14,56.62,-79.48,621.97,1.9592,-0.3632,3.2969,7.2072,5.7062,4.5627,0.226336,3.3277,3.4166,1.8321,0.9979,0.103307,6.8623,5.2963,4.1282,0.219729,8.1381,7.3269,-0.1909,1,-0.58,-0.540,-0.572,-0.3985,-5.18,13,2.89,-1.73,1.04,0.2521,0.7258,1,0,5,0
3,4,134,10,0.439874,1100,3,82642,7542.0,1730.0,132.94,133.58,7,0.8017,0.7881,704687,0.7881,0.7881,0.5923,0.7881,0.8230,0.8158,1.472752e+09,93,55,1115721.0,93.0,93.0,114.63,102.92,93.0,0.9688,0,0,0,4,0,0,0.77405,0.78929,0.750675,...,3.75,5,36.31,18.15,429.81,2.5883,-0.8812,1.2950,2.5777,2.0830,1.6899,0.054630,1.3748,1.3421,0.7982,0.4810,0.081205,2.5571,2.0593,1.6653,0.056470,3.2516,3.0631,-0.1770,1,-0.75,-0.635,-0.745,-0.5100,-2.04,4,1.29,-0.89,0.66,0.2498,0.7119,1,0,5,0
4,5,109,9,0.502749,2900,4,79124,89.0,491.0,122.72,112.77,6,0.5263,0.5263,51985,0.5263,0.5263,0.3044,0.5405,0.5556,0.5455,1.442916e+09,60,21,536400.0,60.0,60.0,60.00,60.00,60.0,0.8451,0,0,0,26,0,0,0.60315,0.53367,0.696150,...,11.45,27,2206.14,-10559.05,708436.49,9.0293,-0.8760,6.3852,17.2599,14.2795,11.9179,0.085330,7.2175,6.2262,3.1446,1.6149,0.074286,15.9080,12.5688,9.9844,0.067540,17.5561,15.6079,-0.4444,1,-0.18,-0.280,-0.182,-0.4277,-11.12,26,6.11,-3.82,2.51,0.2282,-0.5399,0,0,5,0
